### Import library 

In [1]:
import torch
import pandas as pd
import PIL
import matplotlib.pylab as plt

### Visdom

In [2]:
import visdom
import numpy as np
vis = visdom.Visdom(env='ch2')


### Pytorch

In [3]:
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets,models

import torch.nn as nn
import torch.nn.functional as F
import torch

### Torch Vision

In [4]:
feature_extract = models.densenet121(pretrained=True)

/home/jimmyyoung/.local/lib/python3.5/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [5]:
def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm
root_list = ['HMB_1/','HMB_2/','HMB_4/','HMB_5/','HMB_6/']

hbm_1_data = pd.read_csv('./HMB_1/interpolated.csv')
hbm_2_data = pd.read_csv('./HMB_2/interpolated.csv')
hbm_4_data = pd.read_csv('./HMB_4/interpolated.csv')
hbm_5_data = pd.read_csv('./HMB_5/interpolated.csv')

hbm_6_data = pd.read_csv('./HMB_6/interpolated.csv')
df_list = [hbm_1_data,hbm_2_data,hbm_4_data,hbm_5_data,hbm_6_data]
df_center_list = []
for i,df in enumerate(df_list):
    a = df.groupby('frame_id').get_group('center_camera')
#     a['angle'] = (a['angle'])
    a.sort_values('timestamp')
    a.filename = root_list[i]+a.filename
    df_center_list.append(a)
    

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
# video = []
# for i in range(100):
#     video.append(plt.imread(df_center_list[1].iloc[i].filename))
# video = np.array(video)

In [7]:
# vis.video(video)


In [8]:
df_center_list[1]['angle'].max()

1.90326191229

In [9]:
class PARSER():
    def __init__(self):
        self.n_epochs = 10
        self.n_iters = 2000
        self.batch_size = 64
        self.lr = 0.0001
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.latent_dim = 62
        self.img_size = 64
        self.channels = 1
        self.sample_interval = 10
        self.clip_value = 0.01
opt = PARSER()




### Cuda

In [10]:
cuda = True if torch.cuda.is_available() else False
device = 'cuda' if cuda else 'cpu'


### Weight Norm

In [11]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [12]:
# Policy Network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1_0 = nn.Sequential(
            nn.Conv3d(3, 32, (3, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1)),
            nn.ReLU(),
        )
        
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            
        )
        self.maxpool2_0 = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_features=32,out_features=2)
        
#         self.conv4 = nn.Conv2d(32,1,3,1,1)
            
    def forward(self, x):
        
        x = F.relu(self.conv1_0(x)) 
        x = x.squeeze(2)
        x = self.conv2(x)
        x = self.maxpool2_0(x)
        x = F.hardtanh(self.fc(x.view(-1,32)),min_val=-2., max_val=2.)
#         x = F.sigmoid(self.conv4(x))
        return x
# Reward function   
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1_0 = nn.Sequential(
            nn.Conv3d(3, 32, (3, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1)),
            nn.ReLU(),
        )
        self.conv = nn.Sequential(
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.InstanceNorm2d(32),
            nn.LeakyReLU(),
            
        )
        self.fc1 = nn.Linear(8*10*32+2,100)
        self.fc2 = nn.Linear(100,1)
        self.Htanh = torch.nn.Hardtanh()
        

    def forward(self, img,predict):
        out = self.conv1_0(img)
        out = out.squeeze(2)

        out = self.conv(out)
        out = out.view(-1,8*10*32)

        out = torch.cat([out,predict],1)
        
        out = self.fc1(out)
        out = self.fc2(F.relu(out))
        
        return self.Htanh(out)

    
Policy_net = nn.DataParallel(Generator())
Reward_net = nn.DataParallel(Discriminator())
weights_init_normal(Policy_net)
weights_init_normal(Reward_net)
Policy_net,Reward_net = Policy_net.to(device),Reward_net.to(device)

In [13]:
# a = Reward_net(torch.ones((32,3,3,480,640)).to(device),torch.rand((32,3)).to(device))
# a.shape

In [14]:
df_center_list[1]['torque'].max()

2.43912960972

### Transform

In [15]:
transform = transforms.Compose([
    
    transforms.ColorJitter(0.1,0.1,0.1,0.1),
    transforms.ToTensor(),
    
])

### Sample image randomly for five image

In [16]:
# img_batch = []
# label_batch = []
# for batch in range(32):
#     img_list = []
#     label = []
#     rand_hbm = np.random.randint(5)
#     rand = np.random.randint(len(df_center_list[rand_hbm])-3)
#     for i in range(3):
#         a = PIL.Image.open(df_center_list[rand_hbm].iloc[rand+i].filename)
#         img_list.append(transform(a))
#     label_batch.append(torch.from_numpy(df_center_list[rand_hbm].iloc[rand+3][['angle','torque','speed']].values.astype(np.float32)))
#     img_batch.append(torch.stack(img_list))
# torch.stack(img_batch).shape
# torch.stack(label_batch).shape

### Training Progress
random sample continuous 3 images and predict the next frame

In [17]:

optimizer_G = torch.optim.RMSprop(Policy_net.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(Reward_net.parameters(), lr=opt.lr)

In [ ]:
for epoch in range(opt.n_epochs):
    for i in range(opt.n_iters):
        img_batch = []
        label_batch = []
        for batch in range(32):
            img_list = []
            label = []
            rand_hbm = np.random.randint(5)
            rand = np.random.randint(len(df_center_list[rand_hbm])-3)
            for j in range(3):
                a = PIL.Image.open(df_center_list[rand_hbm].iloc[rand+j].filename)
                img_list.append(transform(a))
            label_batch.append(torch.from_numpy(df_center_list[rand_hbm].iloc[rand+3][['angle','torque']].values.astype(np.float32)))
            img_batch.append(torch.stack(img_list))
        img_batch = torch.stack(img_batch).to(device)
        label_batch = torch.stack(label_batch).to(device)
        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_predict = Policy_net(img_batch)
        
        if i%5==0:
            # Loss measures generator's ability to fool the discriminator
            g_loss = -torch.mean(Reward_net(img_batch.detach(),gen_predict))
    
            g_loss.backward()
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
#         print(torch.cat((imgs[:,0],gen_imgs),1).size())
        
        d_real = Reward_net(img_batch.detach(),label_batch) 
        d_fake = Reward_net(img_batch.detach(),gen_predict.detach())

        d_loss_real = -torch.mean(d_real)
        d_loss_fake = torch.mean(d_fake)
        d_loss = d_loss_real + d_loss_fake

        d_loss.backward()
        # Clip weights of discriminator
        
        actuallloss = torch.sum(torch.abs(gen_predict-label_batch))
        for p in Reward_net.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)
        optimizer_D.step()
        
        batches_done = epoch * opt.n_iters + i
#         print(epoch * opt.n_iters + i)
        if batches_done % opt.sample_interval == 0:
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f][Actual loss: %f] " % (epoch, opt.n_epochs, i, 2000,
                                                         d_loss.item(), g_loss.item(),actuallloss))
#             vis.images(torch.cat((imgs[:,0],gen_imgs),1)[0,:,None])

In [ ]:
torch.save(Policy_net.state_dict(),'policy.pt')
torch.save(Reward_net.state_dict(),'reward.pt')

### Sample

In [ ]:
img_list = []
label = []
rand_hbm = np.random.randint(5)
rand = np.random.randint(len(df_center_list[rand_hbm])-3)
for i in range(3):
    a = PIL.Image.open(df_center_list[rand_hbm].iloc[rand+i].filename)
    img_list.append(transform(a))
label = torch.from_numpy(df_center_list[rand_hbm].iloc[rand+3][['angle','torque','speed']].values.astype(np.float32))
img_list = torch.stack(img_list).unsqueeze(0).to(device)

In [ ]:
Policy_net(img_list)

In [ ]:
label